In [16]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os
import json
import datetime

In [17]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [18]:
dataset_name = "SEG_2_retrain"

In [19]:
timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
if not os.path.exists("version"):
    os.makedirs("version")
version_dir = "version/" + timestamp 

timestamp

'20201203-180335'

vocabulary = np.genfromtxt("static/vocabulary.csv", delimiter="\n", dtype=np.int64)
vocab_size = vocabulary.shape[0]
vocab_size

## Tuned Hyper-parameters

import dill

with open("best_hps/ht_4/best_hp_0.pkl", "rb") as f:
    best_hp_0 = dill.load(f)
best_hp_0.values

In [21]:
param_list = dict()

param_list["PAST_HISTORY"] = 16
param_list["FUTURE_TARGET"] = 8
param_list["BATCH_SIZE"] = 128
param_list["EPOCHS"] = 1000         # Early Stopping
param_list["BUFFER_SIZE"] = 200000
param_list["VOCAB_SIZE"] = 16293    #vocab_size

# Tuned HPs
param_list['ENCODER_1_NEURONS'] = 192
param_list['ENCODER_1_DROPOUT'] = 0.1
param_list['ENCODER_2_NEURONS'] = 32
param_list['ENCODER_2_DROPOUT'] = 0.1
param_list['DECODER_1_NEURONS'] = 48
param_list['DECODER_DROPOUT_1'] = 0.2
param_list['DECODER_2_NEURONS'] = 224
param_list['DECODER_DROPOUT_2'] = 0.4
#param_list['LEARNING_RATE'] = 0.001        # Nadam recommends to use default lr

In [22]:
train_set = np.genfromtxt("data/{}_train_set.csv".format(dataset_name), delimiter="\n", dtype=np.int64)

In [23]:
x_train = tf.data.Dataset.from_tensor_slices(train_set[:-param_list["FUTURE_TARGET"]]).window(param_list["PAST_HISTORY"], 1, 1, True)
# As dataset.window() returns "dataset", not "tensor", need to flat_map() it with sequence length
x_train = x_train.flat_map(lambda x: x.batch(param_list["PAST_HISTORY"])) 
x_train = x_train.map(lambda x: tf.one_hot(x, param_list["VOCAB_SIZE"], axis=-1))
x_train = x_train.batch(param_list["BATCH_SIZE"])

In [24]:
y_train = tf.data.Dataset.from_tensor_slices(train_set[param_list["PAST_HISTORY"]:]).window(param_list["FUTURE_TARGET"], 1, 1, True)
y_train = y_train.flat_map(lambda y: y.batch(param_list["FUTURE_TARGET"]))
y_train = y_train.map(lambda y: tf.one_hot(y, param_list["VOCAB_SIZE"], axis=-1))
y_train = y_train.batch(param_list["BATCH_SIZE"])

In [25]:
train_data = tf.data.Dataset.zip((x_train, y_train))#.shuffle(param_list["BUFFER_SIZE"], seed=param_list["SHUFFLE_SEED"])   # shuffle not working

In [26]:
val_set = np.genfromtxt("data/{}_val_set.csv".format(dataset_name), delimiter="\n", dtype=np.int64)

In [27]:
x_val = tf.data.Dataset.from_tensor_slices(val_set[:-param_list["FUTURE_TARGET"]]).window(param_list["PAST_HISTORY"], 1, 1, True)
x_val = x_val.flat_map(lambda x: x.batch(param_list["PAST_HISTORY"]))
x_val = x_val.map(lambda x: tf.one_hot(x, param_list["VOCAB_SIZE"], axis=-1))
x_val = x_val.batch(param_list["BATCH_SIZE"])

In [28]:
y_val = tf.data.Dataset.from_tensor_slices(val_set[param_list["PAST_HISTORY"]:]).window(param_list["FUTURE_TARGET"], 1, 1, True)
y_val = y_val.flat_map(lambda y: y.batch(param_list["FUTURE_TARGET"]))
y_val = y_val.map(lambda y: tf.one_hot(y, param_list["VOCAB_SIZE"], axis=-1))
y_val = y_val.batch(param_list["BATCH_SIZE"])

In [29]:
val_data = tf.data.Dataset.zip((x_val, y_val))

In [32]:
model = keras.models.Sequential()
model.add(keras.layers.Bidirectional(keras.layers.LSTM(param_list["ENCODER_1_NEURONS"], return_sequences=True)))
model.add(keras.layers.Dropout(param_list["ENCODER_1_DROPOUT"]))
model.add(keras.layers.Bidirectional(keras.layers.LSTM(param_list["ENCODER_2_NEURONS"])))
model.add(keras.layers.Dropout(param_list["ENCODER_2_DROPOUT"]))
model.add(keras.layers.RepeatVector(param_list["FUTURE_TARGET"]))
model.add(keras.layers.Bidirectional(keras.layers.LSTM(param_list["DECODER_1_NEURONS"], return_sequences=True)))
model.add(keras.layers.Dropout(param_list["DECODER_DROPOUT_1"]))
model.add(keras.layers.Bidirectional(keras.layers.LSTM(param_list["DECODER_2_NEURONS"], return_sequences=True)))
model.add(keras.layers.Dropout(param_list["DECODER_DROPOUT_2"]))
model.add(keras.layers.TimeDistributed(keras.layers.Dense(param_list["VOCAB_SIZE"], activation='softmax')))

model.compile(optimizer=keras.optimizers.Nadam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [33]:
from IPython.display import clear_output

class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    clear_output(wait = True)

In [34]:
model_history = model.fit(train_data, epochs=param_list["EPOCHS"], validation_data=val_data, callbacks=[keras.callbacks.EarlyStopping('val_loss', patience=5), ClearTrainingOutput()])

1260/1260 [==============================] - 180s 143ms/step - loss: 6.0492 - accuracy: 0.3660 - val_loss: 7.8757 - val_accuracy: 0.2121


In [35]:
model.save(version_dir)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: version/20201203-180335\assets


In [36]:
# Version Issue
model.save(version_dir + "/model.h5")